In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import time
from sklearn import metrics

In [2]:
df = pd.read_csv('./silvio/crawler.csv')

In [3]:
# df = pd.read_pickle('soccerDBNew2.pkl')

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3417 entries, 0 to 3416
Data columns (total 115 columns):
Unnamed: 0                      int64
at_current_draws_away           object
at_current_draws_global         object
at_current_draws_home           object
at_current_loss_away            object
at_current_loss_global          object
at_current_loss_home            object
at_current_wins_away            object
at_current_wins_global          object
at_current_wins_home            object
at_games_without_draw_away      object
at_games_without_draw_global    object
at_games_without_draw_home      object
at_games_without_lose_away      object
at_games_without_lose_global    object
at_games_without_lose_home      object
at_games_without_win_away       object
at_games_without_win_global     object
at_games_without_win_home       object
attacks                         float64
away_team                       object
away_team_score                 int64
city                            obj

In [5]:
df.head()

,Unnamed: 0,at_current_draws_away,at_current_draws_global,at_current_draws_home,at_current_loss_away,at_current_loss_global,at_current_loss_home,at_current_wins_away,at_current_wins_global,at_current_wins_home,...,sub_away_player_3,sub_away_player_4,sub_away_player_5,sub_away_player_6,sub_away_player_7,sub_away_player_8,sub_away_player_9,sub_away_player_10,sub_away_player_11,sub_away_player_12
0,0,-,-,-,-,-,-,-,-,-,...,"('person-32923', 'Rodrigo Mancha')","('person-53474', 'Rodriguinho')","('person-112379', 'Breitner')","('person-9453', 'Zé Love')","('person-40138', 'Maikon Leite')",None,None,None,None,None
1,1,-,-,-,-,-,-,-,-,-,...,"('person-8939', 'Rafael Marques')","('person-2825', 'Fábio Rochemback')","('person-82793', 'Fernando')","('person-8950', 'Joílson')","('person-73190', 'Roberson')",None,None,None,None,None
2,2,-,-,-,-,-,-,-,-,-,...,"('person-71111', 'Caique')","('person-94859', 'Allan')","('person-32699', 'Léo Gago')","('person-33519', 'Rafael Carioca')","('person-20615', 'Jumar')",None,None,None,None,None
3,3,-,-,-,-,-,-,-,-,-,...,"('person-13105', 'Fábio Santos')","('person-140', 'Gilberto')","('person-9112', 'Henrique')","('person-22988', 'J. Guerrón')","('person-63706', 'Thiago Ribeiro')",None,None,None,None,None
4,4,-,-,-,-,-,-,-,-,-,...,"('person-178620', 'Vanderson')","('person-72467', 'Marconi')","('person-70722', 'Kleiton Domingues')","('person-81530', 'Edson Reis')","('person-70899', 'Neto Berola')",None,None,None,None,None


In [6]:
df['result'] = np.nan

In [7]:
len(df)

3417

In [8]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
def getResult(row):
    if row.home_team_score > row.away_team_score:
        result = 'win'
    elif row.home_team_score == row.away_team_score:
        result = 'draw'
    else:
        result = 'lose'
    return result

In [10]:
%timeit df.result = df.apply(getResult, axis=1)

150 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
df[['home_team_score', 'away_team_score', 'result']].head()

,home_team_score,away_team_score,result
0,3,3,draw
1,0,0,draw
2,2,1,win
3,1,2,lose
4,1,0,win


In [12]:
df.shape

(3417, 115)

In [13]:
df.head(3)

,at_current_draws_away,at_current_draws_global,at_current_draws_home,at_current_loss_away,at_current_loss_global,at_current_loss_home,at_current_wins_away,at_current_wins_global,at_current_wins_home,at_games_without_draw_away,...,sub_away_player_4,sub_away_player_5,sub_away_player_6,sub_away_player_7,sub_away_player_8,sub_away_player_9,sub_away_player_10,sub_away_player_11,sub_away_player_12,result
0,-,-,-,-,-,-,-,-,-,-,...,"('person-53474', 'Rodriguinho')","('person-112379', 'Breitner')","('person-9453', 'Zé Love')","('person-40138', 'Maikon Leite')",None,None,None,None,None,draw
1,-,-,-,-,-,-,-,-,-,-,...,"('person-2825', 'Fábio Rochemback')","('person-82793', 'Fernando')","('person-8950', 'Joílson')","('person-73190', 'Roberson')",None,None,None,None,None,draw
2,-,-,-,-,-,-,-,-,-,-,...,"('person-94859', 'Allan')","('person-32699', 'Léo Gago')","('person-33519', 'Rafael Carioca')","('person-20615', 'Jumar')",None,None,None,None,None,win


In [14]:
df.drop(columns=['eventList1st', 'eventList2nd'], inplace=True)

In [38]:
categorical_features=['away_team',
 'city',
 'home_team',
 'referee',
 'stadium',
 'home_player_1',
 'home_player_2',
 'home_player_3',
 'home_player_4',
 'home_player_5',
 'home_player_6',
 'home_player_7',
 'home_player_8',
 'home_player_9',
 'home_player_10',
 'home_player_11',
 'away_player_1',
 'away_player_2',
 'away_player_3',
 'away_player_4',
 'away_player_5',
 'away_player_6',
 'away_player_7',
 'away_player_8',
 'away_player_9',
 'away_player_10',
 'away_player_11',
 'sub_home_player_1',
 'sub_home_player_2',
 'sub_home_player_3',
 'sub_home_player_4',
 'sub_home_player_5',
 'sub_home_player_6',
 'sub_home_player_7',
 'sub_home_player_8',
 'sub_home_player_9',
 'sub_home_player_10',
 'sub_home_player_11',
 'sub_home_player_12',
 'sub_away_player_1',
 'sub_away_player_2',
 'sub_away_player_3',
 'sub_away_player_4',
 'sub_away_player_5',
 'sub_away_player_6',
 'sub_away_player_7',
 'sub_away_player_8',
 'sub_away_player_9',
 'sub_away_player_10',
 'sub_away_player_11',
 'sub_away_player_12']

In [16]:
pd.set_option('display.max_columns', None)

# Dashes to zeros

In [18]:
cols =  ['at_current_draws_away', 
  'at_current_draws_global', 
  'at_current_draws_home', 
  'at_current_loss_away', 
  'at_current_loss_global', 
  'at_current_loss_home', 
  'at_current_wins_away', 
  'at_current_wins_global', 
  'at_current_wins_home', 
  'at_games_without_draw_away', 
  'at_games_without_draw_global', 
  'at_games_without_draw_home', 
  'at_games_without_lose_away', 
  'at_games_without_lose_global', 
  'at_games_without_lose_home', 
  'at_games_without_win_away', 
  'at_games_without_win_global', 
  'at_games_without_win_home', 
  'ht_current_draws_away', 
  'ht_current_draws_global', 
  'ht_current_draws_home', 
  'ht_current_loss_away', 
  'ht_current_loss_global', 
  'ht_current_loss_home', 
  'ht_current_wins_away', 
  'ht_current_wins_global', 
  'ht_current_wins_home', 
  'ht_games_without_draw_away', 
  'ht_games_without_draw_global', 
  'ht_games_without_draw_home', 
  'ht_games_without_lose_away', 
  'ht_games_without_lose_global', 
  'ht_games_without_lose_home', 
  'ht_games_without_win_away', 
  'ht_games_without_win_global', 
  'ht_games_without_win_home' ]

In [19]:
df[cols] = df[cols].apply(lambda x: x.replace('-',0))

In [20]:
df[cols] = df[cols].astype('int')

# Possesion to int

In [21]:
def dropPercent(percent):
    try:
        x = percent.strip('%')
    except:
        x = percent
    return x

In [22]:
df.possession = df.possession.apply(dropPercent)

# DATETIME TO INT

In [23]:
 meses={'abril':'04',
        'maio':'05',
        'junho':'06',
        'julho':'07',
        'agosto':'08',
        'setembro':'09',
        'outubro':'10',
        'novembro':'11',
        'dezembro':'12'}

In [24]:
def monthToNumber(date):
    for month, number in meses.items():
        date = date.replace(month, number)
    return date

In [25]:
df.datetime = df.datetime.apply(monthToNumber)

In [26]:
df.datetime = pd.to_datetime(df.datetime)

In [27]:
def to_integer(dt_time):
    return dt_time.dt.day + 100*dt_time.dt.month + 10000*dt_time.dt.year

In [28]:
%%time
df.datetime = to_integer(df.datetime)

CPU times: user 9.04 ms, sys: 4.04 ms, total: 13.1 ms
Wall time: 11.8 ms


# Round

In [29]:
df['round'].value_counts(dropna=False)

3    360
1    360
7    360
5    360
6    360
4    360
2    359
8    351
9    270
0    269
o      8
Name: round, dtype: int64

In [30]:
df.drop(columns='round', inplace=True)

In [32]:
df.possession = df.possession.astype('float')

In [39]:
def runModel(df, categorical_features):
    t0 = time.time()    
    data = df.drop(columns=['home_team_score','away_team_score','result'])
    label = df['result']
    data = pd.get_dummies(data, dummy_na=True, columns=categorical_features)
    X_train, X_test, y_train, y_test = data[0:2735], data[2735:], label[0:2735], label[2735:]
    params = {
        'max_depth': 10,
        'objective': 'multi:softmax',  # error evaluation for multiclass training
        'num_class': 3,
        'n_gpus': 1
    }
    model = XGBClassifier(param=params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    print ('run time: ', time.time()-t0,'s')
    print("Accuracy:",metrics.accuracy_score(y_test, pred))
    print (cm)

In [40]:
runModel(df, categorical_features)

run time:  505.6239855289459 s
Accuracy: 0.6788856304985337
[[ 75  42  79]
 [ 26 100  36]
 [ 23  13 288]]


In [41]:
df.to_csv('soccerDB3.csv')